In [ ]:
runLocation = "googlecolab" ## local googlecolab

# %%
if runLocation=="googlecolab":
    from google.colab import drive
    drive.mount('/gdrive')
    %cd /gdrive/My\ Drive/Deep\ Classifier\ Trainer

In [ ]:
### Imports

import numpy as np
import cv2 as cv
import random
import pickle
import glob
import os
import shutil
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,AveragePooling2D, Flatten
from sklearn.model_selection import train_test_split
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
!pip install livelossplot
from livelossplot.keras import PlotLossesCallback
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [ ]:
### ALL FUNCTIONS ###


def splitDataset(): ## Split Files for Testing
    traindatafolder = dataPath
    testSplit = test_split_size
    
    ## Make Test Folder if Not Existing
    if not os.path.isdir(traindatafolder[:-1] + "test"):
        os.mkdir(traindatafolder[:-1] + "test")
    else:
        print("Test Folder Existing! Split will add to existing test dataset. Press ENTER to continue...")
        input("")

            
    ## Iterate Per Class
    

    for ctrPath,classPath in enumerate(os.walk(traindatafolder)):
        if ctrPath==0: continue
        thisClassPath = classPath[0]
        thisClass = thisClassPath[len(traindatafolder):]


        ## Create Class Folder if Not Yet Exist
        if not os.path.isdir(traindatafolder[:-1] + "test/" + thisClass):
            os.mkdir(traindatafolder[:-1] + "test/" + thisClass)
    
    
        ## Split For Validation
        
        imgFiles = []
        for ctrext in range(0,len(ext)):
            imgFiles.extend(glob.glob(traindatafolder + thisClass + "/*" + ext[ctrext]))

        
        fileSelectIn = [x for x in range(0,len(imgFiles))]
        tmp = [x for x in range(0,len(imgFiles))]

        fileTrain, fileTest, _ , _ = train_test_split(fileSelectIn,tmp,test_size=testSplit)
        

        for f in fileTest:
            filename = imgFiles[f][len(traindatafolder+thisClass+"/"):-len(ext[0])]

            for ctrext in range(0,len(ext)):
                try:
                    shutil.move(traindatafolder + thisClass + "/" + filename + ext[ctrext],traindatafolder[:-1] + "test/" + thisClass + "/" + filename + ext[ctrext].lower())
                except:
                    pass 
                    # print("Move error: ")
        
 
    print("Finished Splitting.")



def trainModel():
    #from tensorflow.keras.preprocessing import image
    if selectedModel == "vgg16":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.vgg16 import preprocess_input
        sourceModel = keras.applications.VGG16
    elif selectedModel == "inceptionv3":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        sourceModel = keras.applications.InceptionV3
    elif selectedModel == "inceptionv2":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "resnet50":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.resnet50 import preprocess_input
        sourceModel = keras.applications.ResNet50
    elif selectedModel == "mobilenetv2":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        sourceModel = keras.applications.MobileNetV2
    elif selectedModel == "inceptionresnetv2":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "mobilenet":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.mobilenet import preprocess_input
        sourceModel = keras.applications.MobileNet
    elif selectedModel == "xception":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.xception import preprocess_input
        sourceModel = keras.applications.Xception

        
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.models import Model
    from tensorflow.keras import optimizers
    from tensorflow.keras.callbacks import EarlyStopping


    # Adam is an optimization algorithm, replacing the very old Gradient Descent.


    modelName = 'model_' + selectedModel;


    #imports the model and discards the classification layer.
    base_model=sourceModel(weights='imagenet', \
                              include_top=False, \
                              input_shape = (imgsz,imgsz,3)); 


    #Show Architecture
    for i,layer in enumerate(base_model.layers):
        print(i,layer.name,layer.trainable)
        

    ## To Check Last Layer
    #base_model.layers[-1]


    x=base_model.output

    addedLayers = 0;
    # if selectedModel == "vgg16":

    x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
    for neuron in neurons:
      if neuron<1:
        x=Dropout(neuron)(x)
      else:
        x=Dense(neuron,activation='relu')(x) 
      addedLayers = addedLayers + 1
    
    preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;

    # elif selectedModel == "inceptionv3":
    #     x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
    #     preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;
    # elif selectedModel == "resnet50":  
    #     x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
    #     preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;
    # elif selectedModel == "mobilenetv2":  
    #     x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
    #     preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;
        


    model=Model(inputs=base_model.input,outputs=preds)

                      
                  
    for layer in model.layers[0:-(addedLayers+layersToUnfreeze)]:
        layer.trainable = False

    #Show Architecture
    for i,layer in enumerate(model.layers):
        print(i,layer.name,layer.trainable)

        
    ## This datagen is used only for viewing, prior to preprocessing
    viewing_datagent=ImageDataGenerator(rotation_range=rotAngle,
                                    horizontal_flip=flipHorizontal,
                                    vertical_flip=flipVertical,
                                    validation_split=validation_split_size)

    ## Official Training Datagen, PreProcessed
    if rotAngle == 0:
        train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                        horizontal_flip=flipHorizontal,
                                        vertical_flip=flipVertical,
                                        validation_split=validation_split_size) 
    else:
        train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                        rotation_range=rotAngle,
                                        horizontal_flip=flipHorizontal,
                                        vertical_flip=flipVertical,
                                        validation_split=validation_split_size) 

    print("Training Dataset:");
    train_generator=train_datagen.flow_from_directory(dataPath,
                                                    target_size=(imgsz,imgsz),
                                                    color_mode='rgb',
                                                    batch_size=batchSize,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training');

    print("Test Validation Dataset:");
    validation_generator=train_datagen.flow_from_directory(dataPath,
                                                    target_size=(imgsz,imgsz),
                                                    color_mode='rgb',
                                                    batch_size=batchSize,
                                                    class_mode='categorical',
                                                    subset='validation',
                                                    shuffle=True);

    print("Sample Viewing Dataset:");
    ## Only for Viewing Unprocessed Images
    view_generator=viewing_datagent.flow_from_directory(dataPath,
                                                    target_size=(imgsz,imgsz),
                                                    color_mode='rgb',
                                                    batch_size=batchSize,
                                                    class_mode='categorical',
                                                    shuffle=True);

    ## Show 5 Sample Images
    x,y = view_generator.next()
    for i in range(0,min([batchSize,5])):
        img = x[i]
        plt.imshow(img.astype(np.uint8))
        plt.show()
        
                                                      


    adamOpt = optimizers.Adam(lr=learningRate);
    model.compile(optimizer=adamOpt, \
                  loss='categorical_crossentropy', \
                  metrics=['accuracy'])
    # Adam optimizer
    # loss function will be categorical cross entropy
    # evaluation metric will be validation accuracy

    if useAllSamples:
        step_size_train=train_generator.n//train_generator.batch_size
    else:
        step_size_train=fixedStepSize

    if enableEarlyStop:
        es = EarlyStopping(monitor=estype,mode='auto', patience=patience)
        
        history = model.fit_generator(generator=train_generator,
                            validation_data = validation_generator,
                          steps_per_epoch=step_size_train,
                          epochs=maxEpochs,
                          callbacks=[PlotLossesCallback(),es]);
    else:
        
        history = model.fit_generator(generator=train_generator,
                            validation_data = validation_generator,
                          steps_per_epoch=step_size_train,
                          epochs=maxEpochs);





    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()


    ### Save Model
    tf.keras.models.save_model(
        model,
        modelName,
        overwrite=True,
        include_optimizer=True
    )

    #### Save Label Names
    labels = (train_generator.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    print(labels);

    fid = open('label_' + selectedModel + '.sav','wb');
    pickle.dump(labels, fid);
    fid.close();


    print("Model successfully trained and saved.");



### Evaluate Trained Model
def testModel():

    #from tensorflow.keras.preprocessing import image
    if selectedModel == "vgg16":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.vgg16 import preprocess_input
        sourceModel = keras.applications.VGG16
    elif selectedModel == "inceptionv3":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        sourceModel = keras.applications.InceptionV3
    elif selectedModel == "inceptionv2":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "resnet50":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.resnet50 import preprocess_input
        sourceModel = keras.applications.ResNet50
    elif selectedModel == "mobilenetv2":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        sourceModel = keras.applications.MobileNetV2
    elif selectedModel == "inceptionresnetv2":
        imgsz = 299
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "mobilenet":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.mobilenet import preprocess_input
        sourceModel = keras.applications.MobileNet
    elif selectedModel == "xception":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.xception import preprocess_input
        sourceModel = keras.applications.Xception


    
    modelName = "model_" + selectedModel;


    testmodel = tf.keras.models.load_model(
        modelName)
    #    custom_objects=None,
    #    compile=True
    #)

    f = open("label_" + selectedModel + ".sav",'rb');labels = pickle.load(f);f.close();
    labels = list(labels.values());

    X = [];
    y = [];

    
    for label in labels:

        fileList = []
        for ctrext in range(0,len(ext)):
            
            fileList.extend(glob.glob(dataPath[:-1] + "test" + "/" + label + "/*" + ext[ctrext]))
            
        
        
        for thisFile in fileList:
            
            ## CV IMAGE LOAD
            try:
              img = cv.imread(thisFile)
              img = cv.cvtColor(img,cv.COLOR_BGR2RGB);
            
              img = cv.resize(img,(imgsz,imgsz));
              
              img = preprocess_input(img);
            
            except:
              print("There was a problem processing this image:")
              print(thisFile)
              print("Remove the image and try again.")
              return

            X.append(img);
            y.append(labels.index(label));
            
        

    y_preds = testmodel.predict(np.array(X));
    y_pred = [];
    for y_preds_i in y_preds:
        y_pred.append(np.argmax(y_preds_i));
    #y_pred = testmodel.predict(X);

    accuracy = sum(np.equal(y_pred,y)) / len(y) * 100

    print()
    print("RANK 1 Accuracy Against Test Data: " + "{:.2f}".format(accuracy) + " %");


    confmat = confusion_matrix(y, y_pred)

    print()
    print('CONFUSION MATRIX:')
    print(confmat)

    print(labels);

    print();
    print("CLASSIFICATION REPORT:")
    print(classification_report(y, y_pred))





    ## SHOW OUTPUT BREAKDOWN
    print()
    print("OUTPUT DISTRIBUTION:")
    outctr = 0;
    total = sum(sum(confmat));
    for col in confmat.T:
        
        totalcol = sum(col)
        
        print(labels[outctr] + ": " + ("%1.2f") % ((totalcol / total) * 100) + "%")
        
        outctr = outctr + 1;



def testOne():
    #from tensorflow.keras.preprocessing import image
    if selectedModel == "vgg16":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.vgg16 import preprocess_input
        sourceModel = keras.applications.VGG16
    elif selectedModel == "inceptionv3":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        sourceModel = keras.applications.InceptionV3
    elif selectedModel == "inceptionv2":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "resnet50":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.resnet50 import preprocess_input
        sourceModel = keras.applications.ResNet50
    elif selectedModel == "mobilenetv2":
        imgsz = 224; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        sourceModel = keras.applications.MobileNetV2
    elif selectedModel == "inceptionresnetv2":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
        sourceModel = keras.applications.InceptionResNetV2
    elif selectedModel == "xception":
        imgsz = 299; ## 224 resnet and vgg, 299 for inception
        from tensorflow.keras.applications.xception import preprocess_input
        sourceModel = keras.applications.Xception
        
        

    modelName = "model_" + selectedModel;


    testmodel = tf.keras.models.load_model(
        modelName)
    #    custom_objects=None,
    #    compile=True
    #)

    f = open("label_" + selectedModel + ".sav",'rb');labels = pickle.load(f);f.close();
    labels = list(labels.values());

    X = [];
    y = [];


    idx = random.randint(0,len(labels)-1)
    label = labels[idx]

    imgFiles = []
    for ctrext in range(0,len(ext)):
        imgFiles.extend(glob.glob(dataPath[:-1] + "test" + "/" + label + "/*" + ext[ctrext]))

    idxFile = random.randint(0,len(imgFiles)-1)

    filepath = imgFiles[idxFile]
    pos1 = filepath.find("/")
    pos2 = filepath.find("/",pos1+1)
    actualClass = filepath[pos1+1:pos2]

    ## CV IMAGE LOAD
    img = cv.imread(filepath);
    img = cv.cvtColor(img,cv.COLOR_BGR2RGB);


    plt.title("Input Image")
    plt.imshow(img)
    plt.show()
  
    img = cv.resize(img,(imgsz,imgsz));
    img = preprocess_input(img);

    y_pred = testmodel.predict(np.array([img]));
    print("Image Classified as: ",labels[np.argmax(y_pred[0])])
    print("Actual Class:",actualClass)
    print("Classification Score: ",max(y_pred[0]))



In [ ]:
############# TRAINING PARAMETERS ##############

selectedModel = "vgg16"; ## "vgg16" "inceptionresnetv2" "inceptionv3" "resnet50" "mobilenet" "mobilenetv2"
neurons = [4096,0.5,4096,0.5,512] ## Vgg 16: [4096,1024,256] , inceptionresnetv2 [], mobilenet neurons = [1024,256]
dataPath = 'traindatafolder/' ## Folder of Cropped Images

ext = [".jpg",".JPG",".jpeg",".JPEG",".png",".PNG"]
#ext = [".jpeg"]
learningRate = 0.000005; ## VGG: 0.000001 ##inceptionresnetv2 0.000001
test_split_size=0.25
validation_split_size=0.25;
maxEpochs = 1000; 
useAllSamples = False ## True to Consume All Traindata per Epoch (for feature crucial classification like defects)
batchSize = 8 # 128; 
fixedStepSize = 32 ## If useAllSamples is False, get no. steps per epoch
enableEarlyStop = True;
estype = "val_loss"
patience = 25
layersToUnfreeze = 0

## Data Augment Parameters
rotAngle = 0
flipHorizontal = False
flipVertical = False


import os
for root, dirs, files in os.walk(dataPath):
    break;

numClasses = len(dirs);




In [ ]:
##### FUNCTION SEQUENCE #####

In [ ]:
splitDataset()  ### Split the Dataset into Training+Validation and Testing

In [ ]:
trainModel()  ### Train the Deep Model

In [ ]:
testModel()  ## Test with All of Test Dataset and Print Metrics

In [ ]:
testOne()  ## Test With One Random Image from Test Dataset